# Fortran magic's documentation

Fortran magic is an [IPython](http://ipython.org) extension that help to use fortran code in an interactive session. 

It adds a `%%fortran` cell magic that compile and import the Fortran code in the cell, using [F2py](http://wiki.scipy.org/F2py).

The contents of the cell are written to a `.f90`/`.f` file in the
directory `IPYTHONDIR/fortran` using a filename with the hash of the
code. This file is then compiled. The resulting module
is imported and all of its symbols are injected into the user's
namespace.

**Warning**: Starting with `python 3.12`, numpy switched to using `meson` instead of `distutils`. Unfortunately, at the moment, the backward compatibility of `f2py` based on `meson` is incomplete.

* Author: Martín Gaitán <gaitan@gmail.com>
* Homepage: https://github.com/mgaitan/fortran_magic 
* Twitter: [@tin`_`nqn`_`](https://twitter.com/tin_nqn_)
* License: BSD

This software was originally sponsored by [Phasety](http://phasety.com)

Feedback, report of issues and pull requests are welcome!

## Install or upgrade

You can install or upgrade via `pip`

        pip install -U fortran-magic

or install via `conda-forge`

        conda install -c conda-forge fortran-magic

# Usage 

Then you are ready to load the magic 

In [ ]:
%load_ext fortranmagic

To load it each time IPython starts, list it in your configuration file:

    c.InteractiveShellApp.extensions = [
        'fortranmagic'
    ]

In [ ]:
import numpy as np

f_config = "--extra '-DNPY_NO_DEPRECATED_API=0'"

%fortran_config {f_config}

## Basic example

Just mark the cell with `%%fortran` in the first line. The code will be highlighted accordingly and compiled when the cell is run 

In [ ]:
%%fortran

subroutine f1(x, y, z)
    real, intent(in) :: x,y
    real, intent(out) :: z

    z = sin(x+y)

end subroutine f1

In [ ]:
%precision %g
f1(1.0, 2.1415)

In [ ]:
print(f1.__doc__)

In [ ]:
print(f1.__source__)

## Verbosity

By default the magic only returns output when the compilation process fails. But you can increase the verbosity with the flag `-v`

In [ ]:
%%fortran -v

module hi
  integer :: five = 5
end module   

In [ ]:
assert hi.five == 5

In [ ]:
%%fortran -vv

module hi
  integer :: five = 5
end module   

In [ ]:
assert hi.five == 5

In [ ]:
%%fortran -vvv

module hi
  integer :: five = 5
end module   

In [ ]:
assert hi.five == 5

## Using f2py options

Almost all f2py's command line options are exposed to the `%%fortran` cell magic. See the docstring for detail. For example:

In [ ]:
%%fortran -v --f77flags="-ffixed-form" 
C
      SUBROUTINE ZADD(A,B,C,N)
C
      DOUBLE COMPLEX A(*)
      DOUBLE COMPLEX B(*)
      DOUBLE COMPLEX C(*)
      INTEGER N
      DO 20 J = 1, N
         C(J) = A(J)+B(J)
 20   CONTINUE
      END

In [ ]:
print(zadd.__doc__)

In [ ]:
a = np.arange(10, dtype=np.cdouble)
b = a * complex(0, 1)
c = np.empty_like(a)
zadd(a, b, c, len(c))
print(c)

## Linking resources

Use `--link` option. This is `--link-<resource>` in f2py command line

In [ ]:
%%fortran --link lapack -vv

subroutine solve(A, b, x, n)
    ! solve the matrix equation A*x=b using LAPACK
    implicit none

    real*8, dimension(n,n), intent(in) :: A
    real*8, dimension(n), intent(in) :: b
    real*8, dimension(n), intent(out) :: x

    integer :: i, j, pivot(n), ok

    integer, intent(in) :: n
    x = b

    ! find the solution using the LAPACK routine SGESV
    call DGESV(n, 1, A, n, pivot, x, n, ok)
    
end subroutine

In [ ]:
A = np.array([[1, 2.5], [-3, 4]])
b = np.array([1, 2.5])

In [ ]:
print(solve.__doc__)

Which is, by the way, the same than

In [ ]:
x = np.linalg.solve(A, b)
x

In [ ]:
np.testing.assert_allclose(x, solve(A, b))

## Extra arguments

F2py could have many other arguments. You could append extra arguments with `--extra`. For example:


      %%fortran --extra '-L/path/to/open/ -lopenblas'
      
      %%fortran --extra '-D<define> -U<name>'
   
      %%fortran --extra '-DPREPEND_FORTRAN -DUPPERCASE_FORTRAN'
      
      %%fortran --extra '-DNPY_NO_DEPRECATED_API=0'
      
The option `--extra` could be given multiple times.

## Compilers runtime compatibility

Incompatibility of compilers or runtime libraries that python was built with, and which are used to build the python extension, can lead to errors during build and/or errors in loading the resulting python extension module.

For example, at the moment, Visual Studio compiler and GNU Fortran (`gfortran`, formerly `g95`) are not compatible when used with `numpy.f2py`. GNU Fortran is compatible with the mingw32 compiler (32-bit or 64-bit), which is available in `conda-forge` or `MSYS2`.

Detailed description see:
- Numpy issue [Can't import module created by f2py "ImportError: DLL load failed" #16416](https://github.com/numpy/numpy/issues/16416#issue-626668211) ;
- Numpy documentation PR [DOC: Windows and F2PY #20311](https://github.com/numpy/numpy/pull/20311/files) .

## Save options

By default, `%%fortran` call to `f2py` without parameters (except, of course, the `-m` and `-c` needed to compile a new module). You can change this behaviour with `%fortran_config`. This line magic can be used in three different ways:


    %fortran_config

        Show the current custom configuration

    %fortran_config --defaults

        Delete the current configuration and back to defaults

    %fortran_config <other options>

        Save (persitently) <other options> to use with %%fortran. The same arguments allowed for `%%fortran` are available
        
For example, to set the highest verbose level (`-vvv`) as default:

In [ ]:
%fortran_config -vvv {f_config}

Now the use of `%%fortran` will include `-vvv` implicitly

In [ ]:
%%fortran

module hi
  integer :: five = 5
end module   

We can see whatever the default config has

In [ ]:
%fortran_config

You can override that global configuration for one specific cell. For example, `%%fortran -v` will change the verbose level but keep the saved defaults.

In [ ]:
%%fortran -v

module hi
  integer :: five = 5
end module   

To clear the custom defaults and back to the defaults (no arguments) use:

In [ ]:
%fortran_config --defaults

In [ ]:
%fortran_config {f_config}

## Help on f2py 

F2py has some flag that output help. See the docstring of `%f2py_help`

In [ ]:
%f2py_help --link blas

In [ ]:
%f2py_help --resources

In [ ]:
%f2py_help --link lapack

## Cache advanced

As described above, build is performed in a separate directory. Since name of temporary module is obtained as a hash of the source code, compilation keys and configuration, usually when working with a single notebook, this does not cause problems. However, in difficult cases: parallel work with several notebooks, significant dependence of the assembly on environment variables, etc., conflicts may arise over the names of temporary modules. For example, Windows locks writing to files of already loaded modules, Linux and macOS ignore changes when reloading the extension module into the same process, etc.

Identical modules are not rebuild, previously loaded instances are used.

In [ ]:
original_f1 = f1

In [ ]:
%%fortran

subroutine f1(x, y, z)
    real, intent(in) :: x,y
    real, intent(out) :: z

    z = sin(x+y)

end subroutine f1

In [ ]:
import sys

secondary_f1 = f1
assert original_f1.__source__ == secondary_f1.__source__
assert sys.version_info.major < 3 or original_f1.__hash__() == secondary_f1.__hash__()  # or .__code__ compare

<br>
`--add-hash=<str>` flags changes hash and, accordingly, name of temporary module.

In [ ]:
%%fortran --add-hash=1

subroutine f1(x, y, z)
    real, intent(in) :: x,y
    real, intent(out) :: z

    z = sin(x+y)

end subroutine f1

In [ ]:
third_f1 = f1
assert original_f1.__source__ == third_f1.__source__
assert sys.version_info.major < 3 or original_f1.__hash__() != third_f1.__hash__()

<br>
Clearing cache directory.

In [ ]:
%fortran_config -v --clean-cache

<br>
At the moment, when using additional files or libraries, absolute paths should be used.

In [ ]:
%%file tfone.f90

integer function tfone()
    tfone = 1
end

In [ ]:
import os

tfone = os.path.abspath("tfone.f90")

In [ ]:
%%fortran --extra {tfone}

integer function tfdigits()
    real x
    tfdigits = digits(x)
end

In [ ]:
assert tfone() == 1 and tfdigits() == 24

---------------
# Final
* Bugs? Ideas? [Open an issue](https://github.com/mgaitan/fortran_magic)
* Do you want to collaborate? [Fork it](https://github.com/mgaitan/fortran_magic/fork)! and send a pull-request

## Cell tags
Tags          | Descriptions
:-------------|:-------------
`random`      | Tests don't check outpus tagged cells.
`random_long` | `Clear Outputs` before commit. 
`fast`        | Will be tested on `pytest -m 'fast'`
`slow`        | Don't tested on `pytest -m 'not slow'` (pytest-astropy plugin: `pytest --run-slow`)
`skip`, `skip_darwin`, `skip_linux`, `skip_win32`| Skip cell (tests don't compute)
`xfail`, `xfail_darwin`, `xfail_linux`, `xfail_win32`| Cell compute, but failed.


